# <span style='color:OrangeRed'>Illustrative Code Example (Energy Cost Minimization Use Case)</span>

To complement the overview on the pycity_scheduling package's architecture and functionalities, this Jupyter Notebook interactively demonstrates basic capabilities and features of the framework. To this aim, we separate the core workflow of our software into three illustrative code examples which are in line with the different framework components "scenario setup component", "power scheduling component", and "post-processing component". In this context, the following three illustrative code examples represent a simple optimization-based energy cost minimization use case for the day-ahead planning of assets inside a multi-energy system.

## <span style='color:Gray'>#1 Scenario setup code example</span>

The Python code below illustrates the typical workflow for the initial scenario setup step of the desired use case. According to this code, the user must always import all required modules from the pycity_scheduling framework and other third-party modules first, compare lines 1-3. Next and according to lines 5-8, the user must define an (pycity_base) Environment object to be used by the subsequent multi-energy system setup and modelling steps. The Environment object maintains general data, which is valid for all framework objects and which contains time, weather, and/or energy market price data information. For this reason, all objects in pycity_base/pycity_scheduling usually point to an Environment. In this example, we define our Timer object to maintain historical time data for one particular day, which is the 15th March of 2018, and hence we choose a time horizon of 24h with a hourly time discretisation, i.e., 3600 seconds. For the location of our multi-energy system, we instantiate the Weather object with the given coordinates for the city of Aachen, Germany. The Price object is instantiated without optional arguments, which makes the pycity_scheduling framework to automatically load historical energy market price data for Germany.

In lines 10-12, the user can now instantiate and define the different assets and load components that are part of the multi-energy system under investigation. For the sake of exemplification, we define a FixedLoad object with an annual electrical energy demand of 3000kWh/a. The parameter profile_type is set to "H0", which refers the fixed load (i.e., the inflexible load) to follow the standard load profile characteristics of a residential single-family house. Further, we instantiate a Photovoltaic object as well as a Battery object in this example, which represent a photovoltaic unit of peak power 6kWp and a battery storage system of capacity 8.4kWh and charging/discharging power rate of 3.6kW, respectively. In the same way, other assets and loads present in the considered multi-energy system setup could be instantiated and added by the user.

For demonstration purposes, we visualize some of the time series data obtained by the instantiated objects for the 15th March of 2018 in lines 14-25. As it can be seen, we can easily access these time series data by using predefined attributes of the different objects, such as p.da_prices representing the energy spot market day-ahead prices, fi.p_el_schedule representing the fixed load's power demand, and pv.p_el_supply representing the photovoltaic unit's power generation over time.

In [ ]:
import matplotlib.pyplot as plt
from pycity_scheduling.classes import *
from pycity_scheduling.algorithms import *

In [ ]:
t = Timer(op_horizon=24, step_size=3600, initial_date=(2018, 3, 15), initial_time=(0, 0, 0))
w = Weather(timer=t, location=(50.76, 6.07))
p = Prices(timer=t)
e = Environment(timer=t, weather=w, prices=p)

In [ ]:
fi = FixedLoad(environment=e, method=1, annual_demand=3000.0, profile_type="H0")
pv = Photovoltaic(environment=e, method=1, peak_power=6.0)
ba = Battery(environment=e, e_el_max=8.4, p_el_max_charge=3.6, p_el_max_discharge=3.6)

In [ ]:
plot_time = list(range(t.timesteps_used_horizon))
fig, axs = plt.subplots(1, 3)
axs[0].plot(plot_time, p.da_prices, color="black")
axs[0].set_title("Day-ahead energy market prices [ct/kWh]")
axs[1].plot(plot_time, fi.p_el_schedule, color="black")
axs[1].set_title("Single-family house electrical load demand [kW]")
axs[2].plot(plot_time, pv.p_el_supply, color="black")
axs[2].set_title("Residential photovoltaics generation [kW]")
for ax in axs.flat:
    ax.set(xlabel="Time [h]", xlim=[0, t.timesteps_used_horizon-1])
plt.show()

## <span style='color:Gray'>#2 Power scheduling code example</span>

To illustrate the power scheduling workflow step, we extend the scenario setup code example from the previous part. For this purpose, at first we define the hierarchy of our multi-energy setup according to the code stated in lines 27-35. This can be done in a straightforward way, in which we start with the instantiation of a Building object to which one we assign two different EntityContainer subobjects, namely an Apartment object and a BuildingEnergySystem object. The Apartment object takes and maintains energy devices that residents may own and operate on the individual apartment level such as the electrical load and the battery unit in our case, whereas the BuildingEnergySystem object takes and maintains energy devices that are usually installed on the global building level such as the photovoltaic unit.

In the following step, we instantiate a CityDistrict object that can bundle a set of different buildings, but which is only one building in this code example for the sake of exemplification (compare lines 34-35). We further define the CityDistrict object to possess a price-driven optimization objective, as we want to perform an energy cost minimization in this example. However, we could also define the CityDistrict object (and if desired the Building object, too) to aim for an optimization objective other than energy cost instead, such as a peak-shaving or a low CO2 emission objective.

The actual day-ahead power dispatch is performed in lines 37-39. In this step, we pass our CityDistrict object to the pre-available CentralOptimization optimization algorithm in line 37 and then call the Pyomo's underlying third-party optimization solver in line 38. As it can be seen, the mode parameter is set to "integer" in line 37, which makes the pycity_scheduling framework to use a modelling approach based on mixed-integer programming. Lastly, we can (temporally) store the optimal power schedules obtained by the optimization solver by calling the CityDistrict's copy_schedule function in line 39. As it can be seen, we tag those power schedules with "optim_schedule" here.


In [ ]:
bd = Building(environment=e, objective="none")
bes = BuildingEnergySystem(environment=e)
ap = Apartment(environment=e)
bd.addMultipleEntities(entities=[bes, ap])
bes.addDevice(objectInstance=pv)
ap.addMultipleEntities(entities=[fi, ba])

In [ ]:
cd = CityDistrict(environment=e, objective="price")
cd.addEntity(bd, position=(0, 0))

In [ ]:
opt = CentralOptimization(city_district=cd, mode="integer")
res = opt.solve()
cd.copy_schedule(dst="optim_schedule")

## <span style='color:Gray'>#3 Post-processing code example</span>

The following code finalizes the overall workflow of this illustrative code example by demonstrating different post-processing functionalities of the pycity_scheduling framework. For this purpose, in a first step we import the framework's post-processing utilities "metric", "plot_schedules", and "write_schedules" as shown in lines 41-43.

In a second step, we call the CityDistrict's load_schedule function in line 45 to load the previously stored power schedules as tagged with the identifier "optim_schedule".

In a third step, we use the "plot_entity" functionality to plot the schedules of all optimization variables for the two objects of instance CityDistrict and Battery, where the schedules with the suffix "p_el" denote electrical power and "e_el" denote electrical energy, respectively. From the plots it becomes evident that the flexible battery device is scheduled in a way such that power is primarily imported from the energy spot market during cheap tariff periods, compare the price plot from the first part. This means that the battery unit is charged during these periods based on the defined energy cost minimization objective. Because of this behavior, low-cost electrical energy is temporarily stored inside the battery unit. Vice versa, the battery unit is discharged during expensive energy spot market tariff periods to supply the non-flexible building's electrical load locally during these periods. Moreover, one can see that the battery unit is also charged during time slots of high power penetration by the photovoltaic unit, compare the PV plot from the first part. This is because the locally generated photovoltaic energy is assumed to have zero energy costs, i.e., it can be perceived as free. The building's power self-consumption rate metric of approximately 67%, as evaluated in line 50, confirms this circumstance. The remaining 23% of photovoltaic power generation, however, cannot be consumed locally by the building, since the battery unit already operates at its physical charging power limit of 3.6kW. Finally and for further studies, we export the obtained schedules of the different multi-energy system assets into a JSON file named "cost_otpim.json" according to line 52.


In [ ]:
from pycity_scheduling.util.metric import self_consumption
from pycity_scheduling.util.plot_schedules import plot_entity
from pycity_scheduling.util.write_schedules import schedule_to_json

In [ ]:
cd.load_schedule(schedule="optim_schedule")

In [ ]:
plot_entity(entity=cd, schedule=["optim_schedule"], title="City district - Cost-optimal schedules")
plot_entity(entity=ba, schedule=["optim_schedule"], title="Battery unit - Cost-optimal schedules")

In [ ]:
print(self_consumption(entity=bd))

In [ ]:
schedule_to_json(input_list=[fi, pv, ba], file_name="cost_optim.json", schedule=["optim_schedule"])